## Batch Normalization

Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift (Ioffe S., Szegedy C., 2015)

As standard scaling improves learning by normalizing each feature in the training dataset, batch normalization normalizes input to every unit of a hidden layer within a batch. Estimated distribution is over the samples in the batch.
Typically, batch normalization is done before the activation function, but this is not a hard rule.

$$ \textbf{u}^{[l]} = \frac{1}{S} \sum_{s=1}^{S}{\textbf{z}^{[l]}_{s, :}} $$

$$ \textbf{v}^{[l]} = \frac{1}{S} \sum_{s=1}^{S}{(\textbf{z}^{[l]}_{s, :} - \textbf{u}^{[l]})^2} $$

$$ 
\textbf{z}^{[l]} \leftarrow 
\frac 
    {\textbf{z}^{[l]} - \textbf{u}} 
    {\sqrt{\textbf{v}^{[l]} + \epsilon}}
$$

$$
\textbf{z}^{[l]} \leftarrow 
\textbf{z}^{[l]} \odot \tilde{\textbf{v}}^{[l]} + \tilde{\textbf{u}}
$$

Parameters $\tilde{\textbf{v}}^{[l]}$ and $\tilde{\textbf{u}}^{[l]}$ are learnable with back propagation.
Batch normalization of layer $l$ pre-activation makes the input to the layer $l+1$ vary less, since the between-iterations, same-unit distribution's mean and variance are fixed. This improves the speed of learning.

Since estimates of mean and variance are different for each batch, $\textbf{z}^{[l]}$ gets noisy, which also acts as small regularizer. The quantity of noise reduces with the size of the batch.

For validation time, estimate $\textbf{v}^{[l]}$ and $\textbf{u}^{[l]}$ as exponentially weighted average across the training mini-batches. Use these estimates instead of the validation dataset's properties.

### Rerefences
* https://arxiv.org/pdf/1502.03167.pdf
* https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler
* https://github.com/ducha-aiki/caffenet-benchmark/blob/master/batchnorm.md

In [5]:
import numpy as np

m = 10
s = 100
z = np.random.rand(s, m)

u = np.sum(z, axis=0) / s
v = (z - u) ** 2 / s

z = z - u / (np.sqrt(v + 1e-10))

tilde_u = np.random.rand(*u.shape)
tilde_v = np.random.rand(*v.shape)

z = z * tilde_v + tilde_u

print(z.shape)

(100, 10)


In [1]:
import tensorflow.keras.layers
?tensorflow.keras.layers.BatchNormalization

Init signature:
tensorflow.keras.layers.BatchNormalization(
    axis=-1,
    momentum=0.99,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer='zeros',
    gamma_initializer='ones',
    moving_mean_initializer='zeros',
    moving_variance_initializer='ones',
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    renorm=False,
    renorm_clipping=None,
    renorm_momentum=0.99,
    fused=None,
    trainable=True,
    virtual_batch_size=None,
    adjustment=None,
    name=None,
    **kwargs,
)
Docstring:     
Batch normalization layer (Ioffe and Szegedy, 2014).

Normalize the activations of the previous layer at each batch,
i.e. applies a transformation that maintains the mean activation
close to 0 and the activation standard deviation close to 1.

Arguments:
  axis: Integer, the axis that should be normalized
      (typically the features axis).
      For instance, after a `Conv2D` layer with
      `data_form